Solves a 1D problem BVP:
\begin{align}
-u''(x) = f(x)\\
u(0) = 1\\
u(1) = 0
\end{align}

with the choice $f(x) = e^x -2xe^x -x^2e^x$, the solution is given by
\begin{equation}
u(x) = (1-x)^2e^x
\end{equation}

In [15]:
import numpy as np
import matplotlib.pyplot as plt
import numpy.linalg as la

% matplotlib inline

In [16]:
def f(x):
    return np.exp(x)*(1 - 2*x - x**2)

def u(x):
    return np.exp(x)*((1-x)**2)

In [14]:
# use a uniform mesh spacing 
N = 5
x = np.linspace(0,1,N+1)
h = 1/N


Under the partition $0 = x_0,\dots,x_N = 1$, with uniform spacing we want to solve bilinear form:
\begin{equation}
a_{ \epsilon}(u,v) = L_{\epsilon}(v)
\end{equation}
defined as follows:
\begin{equation}
a_{\epsilon} = \sum_{n=0}^{N-1} \int_{x_n}^{x_{n+1}} u'(x)v'(x) dx - \sum_{n=0}^{N} \{u'(x_n)\} [v(x_n)]
+\epsilon \sum_{n=0}^N\{v'(x_n)\} [u(x_n)] + J_0(u,v) + J_1(u,v)
\end{equation}
where 
\begin{align}
J_0(u,v) = \sum_{n=0}^N \frac{\sigma_0}{h}[v(x_n)][u(x_n)] \\
J_1(u,v) = \sum_{n=1}^{N-1}\frac{\sigma_1}{h}[v'(x_n)][u'(x_n)]
\end{align}
and $\epsilon$ takes values of 0, 1, and 2.

\begin{align}
L(v) = \int_0^1 f(x)v(x)dx - \epsilon v'(x_0) + \frac{\sigma_0}{h}v(x_0)
\end{align}
And the jumps and averages are defined at interior nodes (1 through N-1) as:
\begin{align}
[v(x_n)] = (v(x_n^-) - v(x_n^+))\\
\{v(x_n)\} = \frac{1}{2}(v(x_n^+) + v(x_n^-))
\end{align}
At the endpoints we have:
\begin{align}
[v(x_0)] = - v(x_0)\\
[v(x_N)] = v(x_N)\\
\{v(x_0)\} = v(x_0)\\
\{v(x_N)\} = v(x_N)
\end{align}
Assembly of $A$ should be split into 3 parts: Integrals over subintervals, constraints at interior nodes, and constraints at exterior nodes.  Both boundary conditions will be enforced weakly!!!

First, we assemble the local matrix $A_n$ which is the discrete form of the integral over every element:
\begin{equation}
(A_n)_{ij} = \int_{x_n}^{x_{n+1}} \phi_j ' (x) \phi_i '(x) dx
\end{equation}
$A_n$ has the following form for elements 1 through $n$:
\begin{equation}
A_n = \frac{1}{h}\begin{pmatrix}
1 & -1 \\
-1 & 1
\end{pmatrix}
\end{equation}
Every element requires this, so these local matrices will be part of the diagonal of global matrix

Next, we compute contributions to inner nodes.  Expanding $- \sum_{n=0}^{N} \{u'(x_n)\} [v(x_n)]
+\epsilon \sum_{n=0}^N\{v'(x_n)\} [u(x_n)] + J_0(u,v) + J_1(u,v)$ (restricting our selves to interior nodes), we get four types of constraints.  The first is:

\begin{align}
u^+v^+: \frac{1}{2}\phi_j ' (x_n^+)\phi_i(x_n^+) - \frac{\epsilon}{2}\phi_j(x_n^+)\phi_i ' (x_n^+) + \frac{\sigma_0}{h}\phi_j(x_n^+)\phi_i(x_n^+) + \frac{\sigma_1}{h}\phi_j ' (x_n^+) \phi_i ' (x_n^+)
\end{align}
with local matrix
\begin{equation}
B_n = \frac{1}{h} \begin{pmatrix}
\frac{\epsilon}{2} - \frac{1}{2} + \sigma_0 + \frac{\sigma_1}{h^2} & -\frac{\epsilon}{2} - \frac{\sigma_1}{h^2}\\
\frac{1}{2} - \frac{\sigma_1}{h^2} & \frac{\sigma_1}{h^2}
\end{pmatrix}
\end{equation}
These are additional constraints within element $n+1$ (to the right of the interior node).  Subsequently, they will be added to the diagonal, excluding the 1st element (empty upper left-hand block)

2nd type:
\begin{align}
u^-v^-: -\frac{1}{2}\phi_j ' (x_n^-)\phi_i(x_n^-) + \frac{\epsilon}{2}\phi_j(x_n^-)\phi_i ' (x_n^-) + \frac{\sigma_0}{h}\phi_j(x_n^-)\phi_i(x_n^-) + \frac{\sigma_1}{h}\phi_j ' (x_n^-) \phi_i ' (x_n^-)
\end{align}
with matrix:
\begin{equation}
C_n = \frac{1}{h} \begin{pmatrix}
\frac{\sigma_1}{h^2} & \frac{1}{2} - \frac{\sigma_1}{h^2}\\
-\frac{\epsilon}{2} - \frac{\sigma_1}{h^2} & -\frac{1}{2} + \frac{\epsilon}{2} + \sigma_0 + \frac{\sigma_1}{h^2}
\end{pmatrix}
\end{equation}
These are additional constraints within element $n$ (to the left of the interior node).  They will be added to the diagonal, excluding the last element (empty lower right-hand block)

Now we need inter-node coupling.  The first type:
\begin{align}
u^+v^-: -\frac{1}{2}\phi_j ' (x_n^+)\phi_i(x_n^-) - \frac{\epsilon}{2}\phi_j(x_n^+)\phi_i ' (x_n^-) - \frac{\sigma_0}{h}\phi_j(x_n^+)\phi_i(x_n^-) - \frac{\sigma_1}{h}\phi_j ' (x_n^+) \phi_i ' (x_n^-)
\end{align}
with matrix:
D_n = \frac{1}{h} \begin{pmatrix}
\frac{\epsilon}{2} - \frac{\sigma_1}{h^2} & \frac{\sigma_1}{h^2}\\
\frac{1}{2} - \frac{\epsilon}{2} - \sigma_0 + \frac{\sigma_1}{h^2} & -\frac{1}{2} - \frac{\sigma_1}{h^2}
\end{pmatrix}

The overall matrix will have the form:
\begin{equation}
\begin{pmatrix}
A_0 + C_1 & 0 & \dots & \dots & 0\\
0 & A_1 + B_1 + C_2 & 0 & \dots & 0 \\
\vdots & \vdots & \ddots & 0 & \vdots \\
0 & 0 & 0 & A_{N-2} + B_{N-2} + C_{N-1} & 0 \\
0 & 0 & 0 & 0 &A_{N-1} + B_{N-1}
\end{pmatrix}
\end{equation}

In [27]:
# We are going to impose boundary conditions weakly
# We'll do piecewise linears first
# h, sigma_0 and sigma_1 are constant initially

def local_A(h):
    A = np.array([[1.0,-1.0],[-1.0,1.0]])
    A /= h
    return A 

def local_B(h,eps,sig_0,sig_1):
    B11 = eps/2 - 0.5 + sig_0 + sig_1/(h**2)
    
    B12 = -sig_1/(h**2) - eps/2
    
    B21 = 0.5 - sig_1/(h**2)
    
    B22 = sig_1/(h**2)
    B = np.array([[B11,B12],[B21,B22]])
    B /= h
    return B

def interior_up_vm(h,eps,sig_0,sig_1):
    return 1.0
    
    

In [25]:
A = local_stiffness_matrix(h)

In [26]:
A

array([[ 5., -5.],
       [-5.,  5.]])